In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### IOU

In [2]:
def calculate_iou(true_mask, predicted_mask):
    # Convert true mask to gray scale
    true_mask_gray = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    # Convert predicted mask to gray scale
    predicted_mask_gray = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
    
    intersection = np.logical_and(true_mask_gray, predicted_mask_gray)
    union = np.logical_or(true_mask_gray, predicted_mask_gray)

    iou = np.sum(intersection) / np.sum(union)
    return iou

### Dice

In [3]:
def calculate_dice_score(true_mask, predicted_mask):
    # Ensure the masks are binary
    true_mask_binary = true_mask > 0
    predicted_mask_binary = predicted_mask > 0
    
    intersection = np.logical_and(true_mask_binary, predicted_mask_binary)

    dice_score = 0

    # Calculate Dice score only if both true and predicted masks are present
    if np.sum(true_mask_binary) > 0 and np.sum(predicted_mask_binary) > 0:
        # Calculate intersection area
        intersection_area = np.sum(intersection)
        # Calculate area of true mask
        true_area = np.sum(true_mask_binary)
        # Calculate area of predicted mask
        predicted_area = np.sum(predicted_mask_binary)
        
        # Calculate Dice score
        dice_score = 2 * intersection_area / (true_area + predicted_area)
    else:
        dice_score = 0

    return dice_score

### pixel wise accuracy

In [4]:
#pixel wise accuracy

def calculate_pixel_wise_accuracy(true_mask, predicted_mask):
    
    #convert true mask to gray scale
    true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
    #convert predicted mask to gray scale
    predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
    #calculate pixel wise accuracy
    #number of pixels in true mask
    true_positive=np.sum((true_mask_gray==255) & (predicted_mask_gray==255))
    false_positive=np.sum((true_mask_gray==0) & (predicted_mask_gray==255))
    false_negative=np.sum((true_mask_gray==255) & (predicted_mask_gray==0))
    true_negative=np.sum((true_mask_gray==0) & (predicted_mask_gray==0))


    #pixel wise accuracy
    pixel_wise_accuracy=(true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)

    return pixel_wise_accuracy

In [5]:
#dataframe to store the results datset wise
results_df=pd.DataFrame(columns=['Dataset','IOU','Dice Score','Pixel Wise Accuracy'])

true_df =pd.DataFrame(columns=['Dataset','IOU','Dice Score','Pixel Wise Accuracy'])

### Name of folder to eval

In [6]:
folder_name='SAM_predictions_BBox'

#delete benign (195)_mask_2_mask.png image from train/benign folder

benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)
print(list_of_benign_predictions_dir)
#delete benign (195)_mask_2_mask.png image from valid/benign folder
if 'benign (195)_mask_2_mask_prediction.png' in list_of_benign_predictions_dir:
    os.remove(os.path.join(benign_predictions_dir,'benign (195)_mask_2_mask_prediction.png'))

['benign (185)_mask_prediction.png', 'benign (424)_mask_prediction.png', 'benign (422)_mask_prediction.png', 'benign (276)_mask_prediction.png', 'benign (423)_mask_prediction.png', 'benign (355)_mask_prediction.png', 'benign (367)_mask_prediction.png', 'benign (348)_mask_prediction.png', 'benign (315)_mask_prediction.png', 'benign (36)_mask_prediction.png', 'benign (433)_mask_prediction.png', 'benign (394)_mask_prediction.png', 'benign (10)_mask_prediction.png', 'benign (418)_mask_prediction.png', 'benign (199)_mask_prediction.png', 'benign (4)_mask_prediction.png', 'benign (347)_mask_prediction.png', 'benign (317)_mask_prediction.png', 'benign (186)_mask_prediction.png', 'benign (333)_mask_prediction.png', 'benign (173)_mask_prediction.png', 'benign (354)_mask_prediction.png', 'benign (247)_mask_prediction.png', 'benign (232)_mask_prediction.png', 'benign (392)_mask_prediction.png', 'benign (233)_mask_prediction.png', 'benign (179)_mask_prediction.png', 'benign (18)_mask_prediction.pn

In [7]:
def calculate_metrics(benign_gt_dir, benign_predictions_dir,csv_file_name,save_csv=True):
    list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)

    df = pd.DataFrame(columns=['File', 'IOU', 'Dice Score', 'Pixel Wise Accuracy'])

    for files in list_of_benign_predictions_dir:
        # read true mask
        #print(files)
        true_mask = cv2.imread(os.path.join(benign_gt_dir, files.replace('_mask_prediction.png', '_mask.png')))
        # read predicted mask
        predicted_mask = cv2.imread(os.path.join(benign_predictions_dir, files))

        # calculate iou
        iou = calculate_iou(true_mask, predicted_mask)
        # calculate dice score
        dice_score = calculate_dice_score(true_mask, predicted_mask)
        # calculate pixel wise accuracy
        pixel_wise_accuracy = calculate_pixel_wise_accuracy(true_mask, predicted_mask)

        df.loc[len(df)] = [files, iou, dice_score, pixel_wise_accuracy]


    # save the dataframe to a csv file
    if save_csv:
        df.to_csv(csv_file_name, index=False)
    
        print('Metrics saved to ',csv_file_name)
    average_dice_score=np.mean(df['Dice Score'])
    average_iou=np.mean(df['IOU'])
    average_pixel_wise_accuracy=np.mean(df['Pixel Wise Accuracy'])
    
    return average_dice_score,average_iou,average_pixel_wise_accuracy

In [8]:
def calculate_actual_validation_metrics(folder_name,df, bsam_df):
    # calculate metrics for validation dataset
    benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
    benign_predictions_dir = os.path.join('..', folder_name, 'valid', 'benign')
    bsam_benign_predictions_dir = os.path.join('..','..', 'BreastSAM impl',folder_name, 'benign')
    csv_file_name = os.path.join('..',folder_name,'valid','benign_metrics.csv')
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir, benign_predictions_dir,csv_file_name,save_csv=True)
    df.loc[len(df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir, bsam_benign_predictions_dir,bsam_df,save_csv=False)
    bsam_df.loc[len(bsam_df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    
    benign_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','benign')
    benign_predictions_dir = os.path.join('..', folder_name, 'train', 'benign')
    bsam_benign_predictions_dir = os.path.join('..','..', 'BreastSAM impl',folder_name, 'benign')
    csv_file_name = os.path.join('..',folder_name,'train','benign_metrics.csv')
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir, benign_predictions_dir,csv_file_name,save_csv=True)
    df.loc[len(df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir, bsam_benign_predictions_dir,bsam_df,save_csv=False)
    bsam_df.loc[len(bsam_df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    
    malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
    malignant_predictions_dir = os.path.join('..', folder_name, 'valid', 'malignant')
    bsam_malignant_predictions_dir = os.path.join('..','..', 'BreastSAM impl',folder_name, 'malignant')
    csv_file_name = os.path.join('..',folder_name,'valid','malignant_metrics.csv')
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir, malignant_predictions_dir,csv_file_name,save_csv=True)
    df.loc[len(df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir, bsam_malignant_predictions_dir,bsam_df,save_csv=False)
    bsam_df.loc[len(bsam_df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    
    malignant_gt_dir=os.path.join('..','..','Dataset_BUSI_with_GT','malignant')
    malignant_predictions_dir = os.path.join('..', folder_name, 'train', 'malignant')
    bsam_malignant_predictions_dir = os.path.join('..','..', 'BreastSAM impl',folder_name, 'malignant')
    csv_file_name = os.path.join('..',folder_name,'train','malignant_metrics.csv')
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir, malignant_predictions_dir,csv_file_name,save_csv=True)
    df.loc[len(df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir, bsam_malignant_predictions_dir,bsam_df,save_csv=False)
    bsam_df.loc[len(bsam_df)] = [csv_file_name,average_iou,average_dice_score,average_pixel_wise_accuracy]
    
    return df,bsam_df

## only box prompt

In [9]:
results_df

,Dataset,IOU,Dice Score,Pixel Wise Accuracy


In [10]:
results_df,true_df=calculate_actual_validation_metrics(folder_name,results_df,true_df)

Metrics saved to  ../SAM_predictions_BBox/valid/benign_metrics.csv
Metrics saved to  ../SAM_predictions_BBox/train/benign_metrics.csv
Metrics saved to  ../SAM_predictions_BBox/valid/malignant_metrics.csv
Metrics saved to  ../SAM_predictions_BBox/train/malignant_metrics.csv


In [11]:
results_df

,Dataset,IOU,Dice Score,Pixel Wise Accuracy
0,../SAM_predictions_BBox/valid/benign_metrics.csv,0.194016,0.256431,0.921793
1,../SAM_predictions_BBox/train/benign_metrics.csv,0.174441,0.229275,0.908476
2,../SAM_predictions_BBox/valid/malignant_metric...,0.215293,0.307382,0.858512
3,../SAM_predictions_BBox/train/malignant_metric...,0.268226,0.370997,0.870852


In [12]:
true_df

,Dataset,IOU,Dice Score,Pixel Wise Accuracy
0,../SAM_predictions_BBox/valid/benign_metrics.csv,0.775150,0.863927,0.983693
1,../SAM_predictions_BBox/train/benign_metrics.csv,0.775150,0.863927,0.983693
2,../SAM_predictions_BBox/valid/malignant_metric...,0.708367,0.823140,0.954823
3,../SAM_predictions_BBox/train/malignant_metric...,0.708367,0.823140,0.954823


## with box and point prompt

In [13]:
folder_name='SAM_predictions_BBoxPoint'
benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)
print(list_of_benign_predictions_dir)
#delete benign (195)_mask_2_mask.png image from valid/benign folder
if 'benign (195)_mask_2_mask_prediction.png' in list_of_benign_predictions_dir:
    os.remove(os.path.join(benign_predictions_dir,'benign (195)_mask_2_mask_prediction.png'))


['benign (185)_mask_prediction.png', 'benign (424)_mask_prediction.png', 'benign (422)_mask_prediction.png', 'benign (276)_mask_prediction.png', 'benign (423)_mask_prediction.png', 'benign (355)_mask_prediction.png', 'benign (367)_mask_prediction.png', 'benign (348)_mask_prediction.png', 'benign (315)_mask_prediction.png', 'benign (36)_mask_prediction.png', 'benign (433)_mask_prediction.png', 'benign (394)_mask_prediction.png', 'benign (10)_mask_prediction.png', 'benign (418)_mask_prediction.png', 'benign (199)_mask_prediction.png', 'benign (4)_mask_prediction.png', 'benign (347)_mask_prediction.png', 'benign (317)_mask_prediction.png', 'benign (186)_mask_prediction.png', 'benign (333)_mask_prediction.png', 'benign (173)_mask_prediction.png', 'benign (354)_mask_prediction.png', 'benign (247)_mask_prediction.png', 'benign (232)_mask_prediction.png', 'benign (392)_mask_prediction.png', 'benign (233)_mask_prediction.png', 'benign (179)_mask_prediction.png', 'benign (18)_mask_prediction.pn

In [14]:
results_df,true_df=calculate_actual_validation_metrics(folder_name,results_df,true_df)

Metrics saved to  ../SAM_predictions_BBoxPoint/valid/benign_metrics.csv
Metrics saved to  ../SAM_predictions_BBoxPoint/train/benign_metrics.csv
Metrics saved to  ../SAM_predictions_BBoxPoint/valid/malignant_metrics.csv
Metrics saved to  ../SAM_predictions_BBoxPoint/train/malignant_metrics.csv


### Point 

In [15]:
folder_name='SAM_predictions_Point'

#delete benign (195)_mask_2_mask.png image from train/benign folder

benign_predictions_dir=os.path.join('..', folder_name, 'train' ,'benign')
list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)
print(list_of_benign_predictions_dir)
#delete benign (195)_mask_2_mask.png image from valid/benign folder
if 'benign (195)_mask_2_mask_prediction.png' in list_of_benign_predictions_dir:
    os.remove(os.path.join(benign_predictions_dir,'benign (195)_mask_2_mask_prediction.png'))

['benign (185)_mask_prediction.png', 'benign (424)_mask_prediction.png', 'benign (422)_mask_prediction.png', 'benign (276)_mask_prediction.png', 'benign (423)_mask_prediction.png', 'benign (355)_mask_prediction.png', 'benign (367)_mask_prediction.png', 'benign (348)_mask_prediction.png', 'benign (315)_mask_prediction.png', 'benign (36)_mask_prediction.png', 'benign (433)_mask_prediction.png', 'benign (394)_mask_prediction.png', 'benign (10)_mask_prediction.png', 'benign (418)_mask_prediction.png', 'benign (199)_mask_prediction.png', 'benign (4)_mask_prediction.png', 'benign (347)_mask_prediction.png', 'benign (317)_mask_prediction.png', 'benign (186)_mask_prediction.png', 'benign (333)_mask_prediction.png', 'benign (173)_mask_prediction.png', 'benign (354)_mask_prediction.png', 'benign (247)_mask_prediction.png', 'benign (232)_mask_prediction.png', 'benign (392)_mask_prediction.png', 'benign (233)_mask_prediction.png', 'benign (179)_mask_prediction.png', 'benign (18)_mask_prediction.pn

In [16]:
results_df,true_df=calculate_actual_validation_metrics(folder_name,results_df,true_df)

Metrics saved to  ../SAM_predictions_Point/valid/benign_metrics.csv
Metrics saved to  ../SAM_predictions_Point/train/benign_metrics.csv
Metrics saved to  ../SAM_predictions_Point/valid/malignant_metrics.csv
Metrics saved to  ../SAM_predictions_Point/train/malignant_metrics.csv


In [17]:
print(results_df)

                                              Dataset       IOU  Dice Score  \
0    ../SAM_predictions_BBox/valid/benign_metrics.csv  0.194016    0.256431   
1    ../SAM_predictions_BBox/train/benign_metrics.csv  0.174441    0.229275   
2   ../SAM_predictions_BBox/valid/malignant_metric...  0.215293    0.307382   
3   ../SAM_predictions_BBox/train/malignant_metric...  0.268226    0.370997   
4   ../SAM_predictions_BBoxPoint/valid/benign_metr...  0.119093    0.171338   
5   ../SAM_predictions_BBoxPoint/train/benign_metr...  0.115522    0.166025   
6   ../SAM_predictions_BBoxPoint/valid/malignant_m...  0.231074    0.327998   
7   ../SAM_predictions_BBoxPoint/train/malignant_m...  0.267980    0.371381   
8   ../SAM_predictions_Point/valid/benign_metrics.csv  0.122594    0.148693   
9   ../SAM_predictions_Point/train/benign_metrics.csv  0.090008    0.121687   
10  ../SAM_predictions_Point/valid/malignant_metri...  0.259443    0.352432   
11  ../SAM_predictions_Point/train/malignant_metri..

In [18]:
#save csv
results_df.to_csv('resnet150gradcam.csv',index=False)

In [19]:
#ls files in directory

print(os.listdir())

['resnet150gradcam.csv', 'tests_old.ipynb', 'tests_new.ipynb']


In [20]:
print(true_df)

                                              Dataset       IOU  Dice Score  \
0    ../SAM_predictions_BBox/valid/benign_metrics.csv  0.775150    0.863927   
1    ../SAM_predictions_BBox/train/benign_metrics.csv  0.775150    0.863927   
2   ../SAM_predictions_BBox/valid/malignant_metric...  0.708367    0.823140   
3   ../SAM_predictions_BBox/train/malignant_metric...  0.708367    0.823140   
4   ../SAM_predictions_BBoxPoint/valid/benign_metr...  0.790104    0.877165   
5   ../SAM_predictions_BBoxPoint/train/benign_metr...  0.790104    0.877165   
6   ../SAM_predictions_BBoxPoint/valid/malignant_m...  0.715556    0.830464   
7   ../SAM_predictions_BBoxPoint/train/malignant_m...  0.715556    0.830464   
8   ../SAM_predictions_Point/valid/benign_metrics.csv  0.589681    0.677111   
9   ../SAM_predictions_Point/train/benign_metrics.csv  0.589681    0.677111   
10  ../SAM_predictions_Point/valid/malignant_metri...  0.461390    0.588329   
11  ../SAM_predictions_Point/train/malignant_metri..